In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc
import random
import lightgbm as lgb
import re
from sklearn.metrics import *
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
train_err = pd.read_csv('/content/drive/MyDrive/Datacon/ld_data/train_err_data.csv')
train_qual = pd.read_csv('/content/drive/MyDrive/Datacon/ld_data/train_quality_data.csv')
train_prob = pd.read_csv('/content/drive/MyDrive/Datacon/ld_data/train_problem_data.csv')

test_err = pd.read_csv('/content/drive/MyDrive/Datacon/ld_data/test_err_data.csv')
test_qual = pd.read_csv('/content/drive/MyDrive/Datacon/ld_data/test_quality_data.csv')

# create columns

## 1. error type

train

In [ ]:
train_user_id_max = 24999
train_user_id_min = 10000
train_user_number = 15000

id_error = train_err[['user_id','errtype']].values
train_error = np.zeros((train_user_number,42))

for person_idx, err in tqdm(id_error):
    train_error[person_idx - train_user_id_min,err - 1] += 1

train_error.shape

100%|██████████| 16554663/16554663 [00:47<00:00, 347088.49it/s]


(15000, 42)

test

In [ ]:
test_user_id_max = 44998
test_user_id_min = 30000
test_user_number = 14999

id_error = test_err[['user_id','errtype']].values
test_error = np.zeros((test_user_number,42))

for person_idx, err in tqdm(id_error):
    test_error[person_idx - test_user_id_min,err - 1] += 1
test_error = test_error.reshape(test_error.shape[0],-1)
print(test_error.shape)

100%|██████████| 16532648/16532648 [00:47<00:00, 346756.84it/s]

(14999, 42)


## fwver

train

In [ ]:
train_err['fw1'] = train_err.fwver.str.split('.').str[0]
train_err['fw2'] = train_err.fwver.str.split('.').str[1]
train_err['fw3'] = train_err.fwver.str.split('.').str[2]

train_fw = train_err[['user_id', 'fw1', 'fw2', 'fw3']]
fw_desc = train_fw.groupby('user_id').describe()
fw = fw_desc.iloc[:, fw_desc.columns.get_level_values(1)=='top']

train_fw = fw.to_numpy()
train_fw

array([['05', '15', '2138'],
       ['04', '33', '1261'],
       ['05', '15', '2138'],
       ...,
       ['04', '22', '1750'],
       ['04', '22', '1778'],
       ['05', '15', '2138']], dtype=object)

test

In [ ]:
# split fwver to 3 columns
test_err['fw1'] = test_err.fwver.str.split('.').str[0]
test_err['fw2'] = test_err.fwver.str.split('.').str[1]
test_err['fw3'] = test_err.fwver.str.split('.').str[2]

# select the most frquent values in 3 columns
test_fw = test_err[['user_id', 'fw1', 'fw2', 'fw3']]
fw_desc = test_fw.groupby('user_id').describe()
fw = fw_desc.iloc[:, fw_desc.columns.get_level_values(1)=='top']

# conver to numpy array
fw_arr_test = fw.to_numpy()

# 인덱스 맞춰주기
test_fw = np.zeros((14999, 3))
test_fw[fw.index-test_user_id_min] = fw.loc[fw.index]

test_fw.shape

(14999, 3)

## quality

In [ ]:
def get_quality(qual_df, qual_list, min_user_id, user_num, method=None):

    """
    quality value 값 뽑기
    none 값은 ffill 방법으로 채우고, ffill로 채워지지 않은 값은 0으로 채운다

    parameter 설명
    qual_df: quality 값을 뽑아낼 데이터프레임
    qual_list: 사용할 열 리스트
    min_user_id: 가장 작은 user_id(train=10000, test=30000)
    user_num: user 갯수(train=15000, test=14999)
    method: str, 대표값을 뽑을 때 사용하는 특성(eg. sum, max, std, mad, mean 중 1)
    """

    qual_num = qual_df[qual_list].copy()

    # define object columns, 만약 오류나면 obj_col data type list로 바꾸기
    obj_col = qual_num.select_dtypes(include='object').columns

    # object column to numeric column
    for col in obj_col:
        qual_num[col] = qual_num[col].astype(str).str.replace(',', '').astype(float)

    new_qual = qual_df[['user_id', 'fwver', 'time']].join(qual_num)

    # fill missing value
    qual_f = new_qual.sort_values(by=['user_id', 'time']).fillna(method='ffill')
    qual_f = qual_f.fillna(0)

    # method 적용
    if method == 'sum':
        qual_f = qual_f.groupby('user_id').sum().loc[:, qual_list]
        qual = np.zeros((user_num, len(qual_list)))
        qual[qual_f.index-min_user_id] = qual_f.loc[qual_f.index]

    if method == 'max':
        qual_f = qual_f.groupby('user_id').max().loc[:, qual_list]
        qual = np.zeros((user_num, len(qual_list)))
        qual[qual_f.index-min_user_id] = qual_f.loc[qual_f.index]

    if method == 'mean':
        qual_f = qual_f.groupby('user_id').mean().loc[:, qual_list]
        qual = np.zeros((user_num, len(qual_list)))
        qual[qual_f.index-min_user_id] = qual_f.loc[qual_f.index]

    if method == 'mad':
        qual_f = qual_f.groupby('user_id').mad().loc[:, qual_list]
        qual = np.zeros((user_num, len(qual_list)))
        qual[qual_f.index-min_user_id] = qual_f.loc[qual_f.index]

    if method == 'std':
        qual_f = qual_f.groupby('user_id').std().loc[:, qual_list]
        qual = np.zeros((user_num, len(qual_list)))
        qual[qual_f.index-min_user_id] = qual_f.loc[qual_f.index]
        
    if method == None:
        return qual_f

    qual_arr = qual
    return qual_arr

train

In [ ]:
train_qual_sum = get_quality(train_qual, ['quality_'+str(i) for i in [1,2,5,7,8,10,11]], 10000, 15000, method='sum')
train_qual_std = get_quality(train_qual, ['quality_'+str(i) for i in [1,2,5,7,8,10,11]], 10000, 15000, method='std')

test

In [ ]:
test_qual_sum = get_quality(test_qual, ['quality_'+str(i) for i in [1,2,5,7,8,10,11]], 30000, 14999, method='sum')
test_qual_std = get_quality(test_qual, ['quality_'+str(i) for i in [1,2,5,7,8,10,11]], 30000, 14999, method='std')

# quality log count

train

In [ ]:
q_cnt = train_qual.groupby('user_id').count()['time']

train_qcnt = np.zeros(15000)
train_qcnt[q_cnt.index-10000] = q_cnt.loc[q_cnt.index]

test

In [ ]:
q_cnt = test_qual.groupby('user_id').count()['time']

test_qcnt = np.zeros(14999)
test_qcnt[q_cnt.index-30000] = q_cnt.loc[q_cnt.index]

# Problem

In [ ]:
train_y = np.zeros(15000)
train_y[train_prob.user_id.unique()-10000] = 1 
train_y.shape

(15000,)

## Concat

train

In [ ]:
train_x = np.concatenate((train_error, train_fw, train_qual_sum, train_qual_std, train_qcnt[:, None]), axis=1)
train_x.shape

(15000, 60)

test

In [ ]:
test_x = np.concatenate((test_error, test_fw, test_qual_sum, test_qual_std, test_qcnt[:, None]), axis=1)
test_x.shape

(14999, 60)

# test13

In [ ]:
import sklearn
import lightgbm
from lightgbm import LGBMRegressor
import catboost
from catboost import CatBoostRegressor
import xgboost as xgb

In [ ]:
catboost_best_params_ = {'rsm': 0.5,
                          'random_strength': 0.5,
                          'n_estimators': 1000,
                          'min_child_samples': 5,
                          'max_depth': 9,
                          'learning_rate': 0.01,
                          'l2_leaf_reg': 0.001}
lightgbm_best_params_ = {'n_estimators': 1000,
                        'min_child_samples': 5,
                        'max_depth': 10,
                        'learning_rate': 0.01,
                        'lambda_l2': 0,
                        'lambda_l1': 0.0001,
                        'feature_fraction': 0.6}
xgb_best_params_ = {'reg_lambda': 0.1,
                    'reg_alpha': 0.01,
                    'n_estimators': 1000,
                    'min_child_weight': 10,
                    'max_depth': 6,
                    'learning_rate': 0.01}

In [ ]:
cb = CatBoostRegressor(logging_level='Silent', **catboost_best_params_)
model_xgb = xgb.XGBRegressor(**xgb_best_params_)
lgbm = LGBMRegressor(**lightgbm_best_params_)

lgbm.fit(train_x, train_y)
y_pred1 = lgbm.predict(train_x)
print('lgbm model auc score: {0:0.4f}'. format(roc_auc_score(train_y, y_pred1)))

model_xgb.fit(train_x, train_y)
y_pred2 = model_xgb.predict(train_x)
print('XGBoost model auc score: {0:0.4f}'. format(roc_auc_score(train_y, y_pred2)))

cb.fit(train_x, train_y)
y_pred3 = cb.predict(train_x)
print('cb model auc score: {0:0.4f}'. format(roc_auc_score(train_y, y_pred3)))

print('mean value: ', (roc_auc_score(train_y, y_pred1) + roc_auc_score(train_y, y_pred2) + roc_auc_score(train_y, y_pred3)) / 3)

pred_y_list = []
for model in [lgbm, model_xgb, cb]:
    pred_y = model.predict(test_x)
    pred_y_list.append(pred_y.reshape(-1,1))
    
pred_ensemble = np.mean(pred_y_list, axis = 0)
sample_submssion = pd.read_csv('./data/sample_submission.csv')
sample_submssion['problem'] = pred_ensemble.reshape(-1)

sample_submssion.to_csv("test_13.csv", index = False)
pd.read_csv('test_13.csv').head()

[LightGBM] [Warning] lambda_l1 is set=0.0001, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0001
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] lambda_l2 is set=0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
lgbm model auc score: 0.9039
XGBoost model auc score: 0.8970
cb model auc score: 0.9000
mean value:  0.9002924
